In [2]:
import pandas as pd

In [3]:
ruta=r"C:\Users\valetorresr\Downloads\Base NC y SS.xlsx"
NC=pd.read_excel(ruta)

In [4]:
NC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79223 entries, 0 to 79222
Data columns (total 47 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   BASE                        79223 non-null  object        
 1   SS                          79223 non-null  object        
 2   CC                          79223 non-null  object        
 3   OC                          79223 non-null  object        
 4   F12                         79223 non-null  object        
 5   RED                         79223 non-null  object        
 6   4 DIG TARJETA               79223 non-null  object        
 7   FECHA TRANSACCION           79180 non-null  datetime64[ns]
 8   COD AUTORIZACION            79223 non-null  object        
 9   VALOR ORIGINAL DE COMPRA    79222 non-null  object        
 10  TIPO DE DEVOLUCION          79223 non-null  object        
 11  VALOR NUEVA COMPRA          79222 non-null  float64   

In [5]:
procesados=NC[(NC["ESTADO"]=="PROCESADA")|(NC["ESTADO"]=="PROCESADA ")]
procesados=procesados[(procesados["FECHA TRANSACCION"]>="2022-01-01")]
#procesados[(procesados['FECHA']>'2022-01-01')]
#procesados['FECHA TRANSACCION']

In [8]:
procesados["concat"] = procesados["COD AUT NC"].astype(str).str.cat(procesados["4 DIG TARJETA"])
procesados["concat1"] = procesados["COD AUTORIZACION"].astype(str).str.cat(procesados["concat"])
procesados["concat2"] = procesados["MONTO"].astype(str).str.cat(procesados["concat1"])
print(procesados.shape)

(60843, 50)


In [9]:
sinDupl=procesados.drop_duplicates("concat2")
print(sinDupl.shape)

(60841, 50)


In [10]:
agrup=sinDupl[sinDupl.duplicated("concat1",keep=False)]
print(agrup.shape)
agrup['MONTO']=agrup['MONTO'].astype(int)

(4, 50)


C:\Users\valetorresr\AppData\Local\Temp\ipykernel_27352\1603550620.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agrup['MONTO']=agrup['MONTO'].astype(int)


In [11]:
agrup1=agrup.groupby(['COD AUT NC','SS','TIPO DE DEVOLUCION','ESTADO']).sum()
#print(agrup1.shape)
agrup12=agrup1[['MONTO','VALOR DE DEVOLUCION']]
agrup12.reset_index(inplace=True)
agrup12


,COD AUT NC,SS,TIPO DE DEVOLUCION,ESTADO,MONTO,VALOR DE DEVOLUCION
0,1078734736,1-126507950959,PARCIAL,PROCESADA,29900,29900.0
1,1078734736,1-126507950959,TOTAL,PROCESADA,89700,89700.0
2,1081562104,1-133110816274,PARCIAL,PROCESADA,5548500,5548500.0
3,1081562104,1-133331456673,TOTAL,PROCESADA,5548501,5548501.0


In [12]:
sinDupl2=sinDupl.drop_duplicates("concat1")
print(sinDupl2.shape)

(60839, 50)


In [13]:
base=sinDupl2[['COD AUT NC', 'SS','TIPO DE DEVOLUCION','ESTADO','MONTO','VALOR DE DEVOLUCION']]
print(base.shape)

(60839, 6)


In [14]:
FINAL= pd.concat([agrup12, base], sort=False)
print(FINAL.shape)

(60843, 6)


In [15]:
FINAL.to_excel('BASE_NC_2022.xlsx')